***
# 以下のbase_urlと、event_datesを設定し、コードをすべて実行するとCSVが出力されます。

In [ ]:
# ここに気象台を選択した後のURLをコピー%ペーストする
#（例）https://tenmado.app/weatherforecast/岐阜地方気象台/をペーストすると以下のようになる
base_url = 'https://tenmado.app/weatherforecast/%E5%B2%90%E9%98%9C%E5%9C%B0%E6%96%B9%E6%B0%97%E8%B1%A1%E5%8F%B0/' 

# ここにイベント日を複数or一つ挿入する
event_dates = ['2023-02-10','2023-02-01']

***

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from urllib.parse import urljoin, unquote

In [ ]:
def get_weather(base_url, event_dates):

    # 結果を格納するための空のDataFrameを作成
    result_df = pd.DataFrame()

    for event_date in event_dates:
        # 実績日を定義
        performance_date = datetime.strptime(event_date, '%Y-%m-%d')

        # 過去7日間の日付を計算
        dates = [performance_date - timedelta(days=i) for i in range(1, 8)]

        # 各日付に対して処理を実行
        for date in dates:
            first_url = f"{base_url}{date.strftime('%Y%m')}/{date.day}/"
            response = requests.get(first_url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # 次のページへのリンクを取得
            link_tag = soup.find('a', class_='link')
            if link_tag:
                next_url = urljoin(base_url, link_tag['href'])
                response = requests.get(next_url)
                soup = BeautifulSoup(response.text, 'html.parser')
            else:
                continue  # リンクがない場合、処理をスキップ

            # 指定された日付に一致するforecast-target-dateを探す
            forecasts = soup.find_all('div', class_='forecast card card-skin')
            for forecast in forecasts:
                forecast_date = forecast.find('div', class_='forecast-target-date')
                if forecast_date and forecast_date.text.strip() == performance_date.strftime('%m/%d'):
                    # 天気予報と降水確率を取得
                    weather_forecast = forecast.find('div', {'class': 'weather'}).text.strip()
                    pop_num = forecast.find('div', {'class': 'pop-num'}).text.strip()
                    pop_percent = forecast.find('div', {'class': 'pop-percent'}).text.strip()
                    precipitation_probability = f"{pop_num}{pop_percent}"

                    # 気温を取得
                    highest_temperature = forecast.find('div', {'class': 'highest-temperature'}).text.strip()
                    lowest_temperature = forecast.find('div', {'class': 'lowest-temperature'}).text.strip()

                    # 結果をDataFrameに追加
                    new_row = pd.DataFrame({
                        '実績日': [performance_date.date()],
                        '天気予報発表日': [date.date()],
                        '天気予報': [weather_forecast],
                        '降水確率': [precipitation_probability],
                        '最高気温': [highest_temperature],
                        '最低気温': [lowest_temperature]
                    })
                    result_df = pd.concat([result_df, new_row], ignore_index=True).sort_values(['実績日','天気予報発表日'])

    return result_df

In [ ]:
place_name = unquote(base_url.split('/')[-2])
weather = get_weather(base_url, event_dates)
weather.to_csv(f"{place_name}_weather.csv",index=False,encoding="shift_jis",errors="ignore")

In [ ]:
%%writefile get_weather_app.py

# -*- coding: utf-8 -*-
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from urllib.parse import urljoin, unquote
import streamlit as st

def get_weather(base_url, event_dates):

    # 結果を格納するための空のDataFrameを作成
    result_df = pd.DataFrame()

    for event_date in event_dates:
        # 実績日を定義
        performance_date = datetime.strptime(event_date, '%Y-%m-%d')

        # 過去7日間の日付を計算
        dates = [performance_date - timedelta(days=i) for i in range(1, 8)]

        # 各日付に対して処理を実行
        for date in dates:
            first_url = f"{base_url}{date.strftime('%Y%m')}/{date.day}/"
            response = requests.get(first_url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # 次のページへのリンクを取得
            link_tag = soup.find('a', class_='link')
            if link_tag:
                next_url = urljoin(base_url, link_tag['href'])
                response = requests.get(next_url)
                soup = BeautifulSoup(response.text, 'html.parser')
            else:
                continue  # リンクがない場合、処理をスキップ

            # 指定された日付に一致するforecast-target-dateを探す
            forecasts = soup.find_all('div', class_='forecast card card-skin')
            for forecast in forecasts:
                forecast_date = forecast.find('div', class_='forecast-target-date')
                if forecast_date and forecast_date.text.strip() == performance_date.strftime('%m/%d'):
                    # 天気予報と降水確率を取得
                    weather_forecast = forecast.find('div', {'class': 'weather'}).text.strip()
                    pop_num = forecast.find('div', {'class': 'pop-num'}).text.strip()
                    pop_percent = forecast.find('div', {'class': 'pop-percent'}).text.strip()
                    precipitation_probability = f"{pop_num}{pop_percent}"

                    # 気温を取得
                    highest_temperature = forecast.find('div', {'class': 'highest-temperature'}).text.strip()
                    lowest_temperature = forecast.find('div', {'class': 'lowest-temperature'}).text.strip()

                    # 結果をDataFrameに追加
                    new_row = pd.DataFrame({
                        '実績日': [performance_date.date()],
                        '天気予報発表日': [date.date()],
                        '天気予報': [weather_forecast],
                        '降水確率': [precipitation_probability],
                        '最高気温': [highest_temperature],
                        '最低気温': [lowest_temperature]
                    })
                    result_df = pd.concat([result_df, new_row], ignore_index=True).sort_values(['実績日','天気予報発表日'])

    return result_df


area_display = {
    '北海道',
    '東北',
    '関東甲信',
    '北陸',
    '東海',
    '近畿',
    '中国',
    '四国',
    '九州',
    '沖縄'
}

hokkai_display = {
    '稚内地方気象台',
    '旭川地方気象台',
    '網走地方気象台',
    '釧路地方気象台',
    '室蘭地方気象台',
    '札幌管区気象台',
    '函館地方気象台'
}

tohoku_display = {
    '青森地方気象台',
    '盛岡地方気象台',
    '仙台管区気象台',
    '秋田地方気象台',
    '山形地方気象台',
    '福島地方気象台'
}

kanto_display = {
    '水戸地方気象台',
    '宇都宮地方気象台',
    '前橋地方気象台',
    '熊谷地方気象台',
    '銚子地方気象台',
    '東京気象庁',
    '横浜地方気象台',
    '甲府地方気象台',
    '長野地方気象台'
}


hokuriku_display = {
    '新潟地方気象台',
    '富山地方気象台',
    '金沢地方気象台',
    '福井地方気象台'
}

tokai_display = {
    '岐阜地方気象台',
    '静岡地方気象台',
    '名古屋地方気象台',
    '津地方気象台'
}

kinki_display = {
    '彦根地方気象台',
    '京都地方気象台',
    '大阪管区気象台',
    '神戸地方気象台',
    '奈良地方気象台',
    '和歌山地方気象台'
}

chugoku_display = {
    '鳥取地方気象台',
    '松江地方気象台',
    '岡山地方気象台',
    '広島地方気象台',
    '下関地方気象台'
}

shikoku_display = {
    '徳島地方気象台',
    '高松地方気象台',
    '松山地方気象台',
    '高知地方気象台'
}

kyushu_display = {
    '福岡管区気象台',
    '佐賀地方気象台',
    '長崎地方気象台',
    '熊本地方気象台',
    '大分地方気象台',
    '宮崎地方気象台',
    '鹿児島地方気象台'
}

okinawa_display = {
    '沖縄気象台',
    '南大東島地方気象台',
    '宮古島地方気象台',
    '石垣島地方気象台'
}


selected_areas_display = st.multiselect('エリアを選択してください', area_display.keys(), default=None)

if selected_areas_display == '北海道':
    selected_place_display = st.multiselect('詳細エリアを選択してください', hokkai_display.keys(), default=None)
elif selected_areas_display == '東北':
    selected_place_display = st.multiselect('詳細エリアを選択してください', tohoku_display.keys(), default=None)
elif selected_areas_display == '関東甲信':
    selected_place_display = st.multiselect('詳細エリアを選択してください', kanto_display.keys(), default=None)
elif selected_areas_display == '北陸':
    selected_place_display = st.multiselect('詳細エリアを選択してください', hokuriku_display.keys(), default=None)
elif selected_areas_display == '東海':
    selected_place_display = st.multiselect('詳細エリアを選択してください', tokai_display.keys(), default=None)
elif selected_areas_display == '近畿':
    selected_place_display = st.multiselect('詳細エリアを選択してください', kinki_display.keys(), default=None)
elif selected_areas_display == '中国':
    selected_place_display = st.multiselect('詳細エリアを選択してください', chugoku_display.keys(), default=None)
elif selected_areas_display == '四国':
    selected_place_display = st.multiselect('詳細エリアを選択してください', shikoku_display.keys(), default=None)
elif selected_areas_display == '九州':
    selected_place_display = st.multiselect('詳細エリアを選択してください', kyushu_display.keys(), default=None)
elif selected_areas_display == '沖縄':
    selected_place_display = st.multiselect('詳細エリアを選択してください', okinawa_display.keys(), default=None)

url = f'https://tenmado.app/weatherforecast/{selected_place_display}/'

d = st.date_input('誕生日を入力してください。', datetime.date(1998, 2, 16))

df = get_weather(url,b)

    # 統計情報をCSVとしてダウンロードするボタン
csv_stats = df.to_csv(f"{selected_areas_display}_{selected_place_display}_weather.csv",index=False,encoding="shift_jis",errors="ignore")
st.download_button(
    label='天候情報をCSVとしてダウンロード',
    data=csv_stats,
    file_name=f'stats_{datetime.today().strftime("%Y%m%d")}.csv',
    mime='text/csv'
) 

In [1]:
%%writefile get_weather_app.py

# -*- coding: utf-8 -*-
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from urllib.parse import urljoin
import streamlit as st

def get_weather(base_url, event_dates):

    # 結果を格納するための空のDataFrameを作成
    result_df = pd.DataFrame()

    for event_date in event_dates:
        # 実績日を定義
        performance_date = datetime.strptime(event_date, '%Y-%m-%d')

        # 過去7日間の日付を計算
        dates = [performance_date - timedelta(days=i) for i in range(1, 8)]

        # 各日付に対して処理を実行
        for date in dates:
            first_url = f"{base_url}{date.strftime('%Y%m')}/{date.day}/"
            response = requests.get(first_url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # 次のページへのリンクを取得
            link_tag = soup.find('a', class_='link')
            if link_tag:
                next_url = urljoin(base_url, link_tag['href'])
                response = requests.get(next_url)
                soup = BeautifulSoup(response.text, 'html.parser')
            else:
                continue  # リンクがない場合、処理をスキップ

            # 指定された日付に一致するforecast-target-dateを探す
            forecasts = soup.find_all('div', class_='forecast card card-skin')
            for forecast in forecasts:
                forecast_date = forecast.find('div', class_='forecast-target-date')
                if forecast_date and forecast_date.text.strip() == performance_date.strftime('%m/%d'):
                    # 天気予報と降水確率を取得
                    weather_forecast = forecast.find('div', {'class': 'weather'}).text.strip()
                    pop_num = forecast.find('div', {'class': 'pop-num'}).text.strip()
                    pop_percent = forecast.find('div', {'class': 'pop-percent'}).text.strip()
                    precipitation_probability = f"{pop_num}{pop_percent}"

                    # 気温を取得
                    highest_temperature = forecast.find('div', {'class': 'highest-temperature'}).text.strip()
                    lowest_temperature = forecast.find('div', {'class': 'lowest-temperature'}).text.strip()

                    # 結果をDataFrameに追加
                    new_row = pd.DataFrame({
                        '実績日': [performance_date.date()],
                        '天気予報発表日': [date.date()],
                        '天気予報': [weather_forecast],
                        '降水確率': [precipitation_probability],
                        '最高気温': [highest_temperature],
                        '最低気温': [lowest_temperature]
                    })
                    result_df = pd.concat([result_df, new_row], ignore_index=True).sort_values(['実績日','天気予報発表日'])

    return result_df

# エリアと詳細エリアの辞書
area_display = {
    '北海道': {
        '稚内地方気象台',
        '旭川地方気象台',
        '網走地方気象台',
        '釧路地方気象台',
        '室蘭地方気象台',
        '札幌管区気象台',
        '函館地方気象台'
    },
    '東北': {
        '青森地方気象台',
        '盛岡地方気象台',
        '仙台管区気象台',
        '秋田地方気象台',
        '山形地方気象台',
        '福島地方気象台'
    },
    '関東甲信': {
        '水戸地方気象台',
        '宇都宮地方気象台',
        '前橋地方気象台',
        '熊谷地方気象台',
        '銚子地方気象台',
        '東京気象庁',
        '横浜地方気象台',
        '甲府地方気象台',
        '長野地方気象台'
    },
    '北陸': {
        '新潟地方気象台',
        '富山地方気象台',
        '金沢地方気象台',
        '福井地方気象台'
    },
    '東海': {
        '岐阜地方気象台',
        '静岡地方気象台',
        '名古屋地方気象台',
        '津地方気象台'
    },
    '近畿': {
        '彦根地方気象台',
        '京都地方気象台',
        '大阪管区気象台',
        '神戸地方気象台',
        '奈良地方気象台',
        '和歌山地方気象台'
    },
    '中国': {
        '鳥取地方気象台',
        '松江地方気象台',
        '岡山地方気象台',
        '広島地方気象台',
        '下関地方気象台'
    },
    '四国': {
        '徳島地方気象台',
        '高松地方気象台',
        '松山地方気象台',
        '高知地方気象台'
    },
    '九州': {
        '福岡管区気象台',
        '佐賀地方気象台',
        '長崎地方気象台',
        '熊本地方気象台',
        '大分地方気象台',
        '宮崎地方気象台',
        '鹿児島地方気象台'
    },
    '沖縄': {
        '沖縄気象台',
        '南大東島地方気象台',
        '宮古島地方気象台',
        '石垣島地方気象台'
    }
}

st.title('天気予報取得アプリ')

# エリア選択
selected_area = st.selectbox('エリアを選択してください', list(area_display.keys()))

# 詳細エリア選択
if selected_area:
    selected_place = st.multiselect('詳細エリアを選択してください', list(area_display[selected_area]))

# 日付選択
event_dates = st.date_input('日付を選択してください', [])

if st.button('天気予報取得開始'):
    if selected_place and event_dates:
        base_url = 'https://tenmado.app/weatherforecast/'
        urls = [f"{base_url}{place}/" for place in selected_place]

        all_dfs = []
        for url in urls:
            df = get_weather(url, [date.strftime('%Y-%m-%d') for date in event_dates])
            all_dfs.append(df)

        combined_df = pd.concat(all_dfs, ignore_index=True)
        st.write('天気予報取得完了')
        st.dataframe(combined_df)

        csv = combined_df.to_csv(index=False, encoding='shift_jis', errors='ignore')
        st.download_button(
            label='天候情報をCSVとしてダウンロード',
            data=csv,
            file_name=f'weather_{datetime.today().strftime("%Y%m%d")}.csv',
            mime='text/csv'
        )

Writing get_weather_app.py


In [2]:
!echo | streamlit run get_weather_app.py


      Welcome to Streamlit!

      If you’d like to receive helpful onboarding emails, news, offers, promotions,
      and the occasional swag, please enter your email address below. Otherwise,
      leave this field blank.

      Email:  


2024-08-07 10:27:36.269 'utf-8' codec can't decode byte 0x82 in position 5: invalid start byte
